In [1]:
import pandas as pd
import json

In [2]:
# filepath
data_file_path = "/Users/gandalf/Documents/coding/do_not_commit/capstone/"

In [3]:
# read in dataframe
message_filename = data_file_path + "raw_data_messages.json"
DF = pd.read_json(message_filename)
print("... read in dataframe")

... read in dataframe


In [4]:
DF.head()

,conversations,users
-KdmSL0izd_44wrniy-W,"{'-KdmSRA-1a7niRVp5s83': {'read': True, 'text'...",NaN
-KdmixiTP5parpBtLtBJ,"{'-Kdmj-d4TwKFUMQkPKeF': {'read': False, 'text...",NaN
-Ke18G-UGnLrasv4_xXT,"{'-Ke18IPzq_R4mOHswlVh': {'read': True, 'readB...",NaN
-Ke28RHZLNV4m69_08sZ,"{'-Ke28SI_EyKkov9BteUL': {'read': False, 'text...",NaN
-Ke4mWs5IgcfwfRwdRFA,"{'-Ke4mjlJmO0XJUwci1dy': {'read': True, 'times...",NaN


In [5]:
len(DF)

30639

In [5]:
# create a copy
df = DF.copy()
df = df[24:]
print("... created a copy and cropped odd convos from start")

... created a copy and cropped odd convos from start


In [6]:
# split into conversation and last message dataframes
df['conv_flag'] = df.conversations.apply(lambda x: 1 if isinstance(x, dict) else 0)
df = df[df.conv_flag==1].drop(['users','conv_flag'],axis=1)
# users_df = df[df.conv_flag==0].drop(['conversations','conv_flag'],axis=1)
print("... got just conversations")

... got just conversations


In [7]:
# get conversation length
df['len_convo'] = df.conversations.apply(len)
print("... got conversation lengths")

... got conversation lengths


In [8]:
# to get all possible column headings (#oneliners)
master_set = set()
for s in df.conversations.apply(lambda x: set(list(x.values())[0].keys())).tolist(): master_set = master_set.union(s) 

def get_deets_fn(x,label):
    try: return list(x.values())[0][label]
    except: return None
    
column_headings = list(master_set)

for heading in column_headings:
    df['first_message_' + heading] = df.conversations.apply(lambda x: get_deets_fn(x,heading))
print("... got details from the first message")

... got details from the first message


In [9]:
column_headings

['uid', 'emailed', 'text', 'read', 'emailAttempted', 'timestamp', 'imageURL']

In [10]:
# this is really gross, but has a kinda nifty end if you're willing to invest the time

df['indexies'] = df.index
df['first_ten'] = df.indexies.apply(lambda x: x[:10])
df['second_ten'] = df.indexies.apply(lambda x: x[10:])
df['is_first'] = df.first_message_uid == df['first_ten']
df['is_second'] = df.first_message_uid == df['second_ten']
df['second_message_uid'] = df.is_second*df.first_ten + df.is_first*df.second_ten

print("... added second message uid")

... added second message uid


In [11]:
# fill blank conversation with empty strings
df.first_message_text = df.first_message_text.fillna('')

# get len of message text
df['first_message_len_char'] = df.first_message_text.apply(len)
df['first_message_len_word'] = df.first_message_text.apply(lambda x: len(x.split()))
print("... got the length of the first message")

... got the length of the first message


In [12]:
# other cleanup
df.first_message_timestamp = \
    df.first_message_timestamp.apply(lambda x: 
                                           pd.to_datetime(x*1000000) 
                                           if isinstance(x, int) 
                                           else x)
print("... converted timestamp to datetime")

... converted timestamp to datetime


In [13]:
df['first_message_date'] = df.first_message_timestamp.apply(lambda x: x.date())

df['first_message_year'] = df.first_message_timestamp.apply(lambda x: x.year)
df['first_message_day_of_year'] = df.first_message_timestamp.apply(lambda x: int(x.strftime('%j')))

df['first_message_month'] = df.first_message_timestamp.apply(lambda x: x.month)
df['first_message_day_of_month'] = df.first_message_timestamp.apply(lambda x: x.day)

df['first_message_week'] = df.first_message_timestamp.apply(lambda x: x.isocalendar()[1])
df['first_message_day_of_week'] = df.first_message_timestamp.apply(lambda x: x.isoweekday())

df['first_message_day'] = df.first_message_timestamp.apply(lambda x: x.toordinal())
df['first_message_hour'] = df.first_message_timestamp.apply(lambda x: x.hour)

print("... broke down time")

... broke down time


In [14]:
# start droppin
col_to_drop = ['conversations','indexies','first_ten','second_ten','is_first','is_second', 'first_message_emailed',
       'first_message_imageURL', 'first_message_emailAttempted',]
# df = df.drop(col_to_drop,axis=1)
# print("... dropped a bunch of lame columns")

In [15]:
df.head(1)

,conversations,len_convo,first_message_uid,first_message_emailed,first_message_text,first_message_read,first_message_emailAttempted,first_message_timestamp,first_message_imageURL,indexies,...,first_message_len_word,first_message_date,first_message_year,first_message_day_of_year,first_message_month,first_message_day_of_month,first_message_week,first_message_day_of_week,first_message_day,first_message_hour
021QQEg1bF013LzOrVju,"{'-KykPRRd6nQPEATeQwnP': {'read': False, 'text...",1,021QQEg1bF,True,Is this Available,False,True,2017-11-12 13:27:23.250,None,021QQEg1bF013LzOrVju,...,3,2017-11-12,2017,316,11,12,45,7,736645,13


In [16]:
df.columns

Index(['conversations', 'len_convo', 'first_message_uid',
       'first_message_emailed', 'first_message_text', 'first_message_read',
       'first_message_emailAttempted', 'first_message_timestamp',
       'first_message_imageURL', 'indexies', 'first_ten', 'second_ten',
       'is_first', 'is_second', 'second_message_uid', 'first_message_len_char',
       'first_message_len_word', 'first_message_date', 'first_message_year',
       'first_message_day_of_year', 'first_message_month',
       'first_message_day_of_month', 'first_message_week',
       'first_message_day_of_week', 'first_message_day', 'first_message_hour'],
      dtype='object')

In [24]:
# rearrange columns
df = df[['first_message_uid', 'second_message_uid',
                     'len_convo', 'first_message_read',
                     'first_message_timestamp', 
                     'first_message_text', 'first_message_len_char', 'first_message_len_word', 'first_message_date',
                     'first_message_year', 'first_message_day_of_year',
                       'first_message_month', 'first_message_day_of_month',
                       'first_message_week', 'first_message_day_of_week', 'first_message_day',
                       'first_message_hour']]
print("... rearranged remaining columns")

... rearranged remaining columns


In [25]:
df.head(2)

,first_message_uid,second_message_uid,len_convo,first_message_read,first_message_timestamp,first_message_text,first_message_len_char,first_message_len_word,first_message_date,first_message_year,first_message_day_of_year,first_message_month,first_message_day_of_month,first_message_week,first_message_day_of_week,first_message_day,first_message_hour
02sjTzcsaO02omVxM8Xm,02omVxM8Xm,02sjTzcsaO,1,False,2017-09-11 02:41:19.188,Wya?,4,1,2017-09-11,2017,254,9,11,37,1,736583,2
07lVtClAEQ01iDVeaXl1,01iDVeaXl1,07lVtClAEQ,1,False,2017-09-06 20:03:30.419,Hey I'm looking for some roommates too have yo...,62,11,2017-09-06,2017,249,9,6,36,3,736578,20


# LIMIT ANALYSIS TO MESSAGES BETWEEN MAR 1, 2017 and SEPT 30, 2017


In [26]:
df = df[df.first_message_month < 10]
df = df[df.first_message_month > 2]
print(min(df.first_message_timestamp))
print(max(df.first_message_timestamp))
print("... cropped date to Jan through Sept 2017")

2017-03-01 00:30:30.688000
2017-09-30 23:59:45.970000
... cropped date to Jan through Sept 2017


# Add Constant for the lols

In [27]:
df['const2'] = 1
print("... added constant")

... added constant


# ADD RESPONSE COLUMN

In [28]:
df['response'] = df.len_convo.apply(lambda x: 1 if x > 1 else 0)

# SAVE AS JSON

In [29]:
df.to_json(data_file_path+'convo_df.json')
print("... saved as json")

... saved as json


# SAVE AS PICKLE

In [30]:
df.to_pickle(data_file_path + 'convo_df.pkl')
print("... saved as pickle")

... saved as pickle


In [31]:
df.head()

,first_message_uid,second_message_uid,len_convo,first_message_read,first_message_timestamp,first_message_text,first_message_len_char,first_message_len_word,first_message_date,first_message_year,first_message_day_of_year,first_message_month,first_message_day_of_month,first_message_week,first_message_day_of_week,first_message_day,first_message_hour,const2,response
02sjTzcsaO02omVxM8Xm,02omVxM8Xm,02sjTzcsaO,1,False,2017-09-11 02:41:19.188,Wya?,4,1,2017-09-11,2017,254,9,11,37,1,736583,2,1,0
07lVtClAEQ01iDVeaXl1,01iDVeaXl1,07lVtClAEQ,1,False,2017-09-06 20:03:30.419,Hey I'm looking for some roommates too have yo...,62,11,2017-09-06,2017,249,9,6,36,3,736578,20,1,0
0AFU6guKDt01iDVeaXl1,01iDVeaXl1,0AFU6guKDt,1,True,2017-09-06 20:04:10.818,Hey I'm interested,18,3,2017-09-06,2017,249,9,6,36,3,736578,20,1,0
0AFU6guKDt01mfWG4vUm,0AFU6guKDt,01mfWG4vUm,1,False,2017-07-03 20:14:06.271,"Check out my profile, let me know if you're in...",54,10,2017-07-03,2017,184,7,3,27,1,736513,20,1,0
0AFU6guKDt06y5iMXoCP,0AFU6guKDt,06y5iMXoCP,1,False,2017-07-03 20:18:55.419,Hey!! Check out my profile & let me know if yo...,66,13,2017-07-03,2017,184,7,3,27,1,736513,20,1,0
